# Loading the Linear regression with minibatch gradient descent model

In [50]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

loading data:

In [51]:
housing=fetch_california_housing()

scaling data (important in case of gradient descent):

In [52]:
m, n=housing.data.shape
stdsc=StandardScaler()
scaled_housing=stdsc.fit_transform(housing.data)
scaled_housing_plus_bias=np.c_[np.ones([m,1]),scaled_housing]

scaled_housing_plus_bias.mean(axis=0)

array([ 1.00000000e+00,  6.60969987e-17,  5.50808322e-18,  6.60969987e-17,
       -1.06030602e-16, -1.10161664e-17,  3.44255201e-18, -1.07958431e-15,
       -8.52651283e-15])

batchsize and number of batches to be processed in one epoch:

In [53]:
batchsize=100
n_batches=np.ceil(m/batchsize).astype(int)
n_batches

207

function for selection of a batch and parsing X, y:

In [54]:
def select_minibatch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches+batch_index)
    #print(epoch*n_batches+batch_index)
    indices=np.random.randint(m, size=batch_size)
    X=scaled_housing_plus_bias[indices]
    y=housing.target[indices]
    #print(indices)
    return X,y

X,y=select_minibatch(1, batchsize, 5)
X.shape

(5, 9)

computational graph of one gradient descent step:

In [55]:
tf.reset_default_graph()
importer=tf.train.import_meta_graph("./saved_models/final_minibatch.ckpt.meta")

In [56]:
gf=tf.get_default_graph()
gf.get_operations()

[<tf.Operation 'save/RestoreV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/RestoreV2/tensor_names' type=Const>,
 <tf.Operation 'save/SaveV2/shape_and_slices' type=Const>,
 <tf.Operation 'save/SaveV2/tensor_names' type=Const>,
 <tf.Operation 'save/Const' type=Const>,
 <tf.Operation 'save/RestoreV2' type=RestoreV2>,
 <tf.Operation 'Training_op/learning_rate' type=Const>,
 <tf.Operation 'gradients/mse_grad/Maximum/y' type=Const>,
 <tf.Operation 'gradients/mse_grad/Const_1' type=Const>,
 <tf.Operation 'gradients/mse_grad/Const' type=Const>,
 <tf.Operation 'gradients/mse_grad/Shape_2' type=Const>,
 <tf.Operation 'gradients/mse_grad/Prod_1' type=Prod>,
 <tf.Operation 'gradients/mse_grad/Maximum' type=Maximum>,
 <tf.Operation 'gradients/mse_grad/Reshape/shape' type=Const>,
 <tf.Operation 'gradients/grad_ys_0' type=Const>,
 <tf.Operation 'gradients/Shape' type=Const>,
 <tf.Operation 'gradients/Fill' type=Fill>,
 <tf.Operation 'gradients/mse_grad/Reshape' type=Reshape>,
 <tf.Operation '

Runing gradient descent step graph several times updating the variable theta:

In [64]:
#init=tf.global_variables_initializer()
with tf.Session() as sess:
    importer.restore(sess, "./saved_models/final_minibatch.ckpt")
    theta = tf.get_default_graph().get_tensor_by_name("Theta:0")
    training_op = tf.get_default_graph().get_tensor_by_name("Training_op:0")
    #sess.run(init)
    #print(theta.eval())
    n_epochs=10
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            Xb,yb=select_minibatch(epoch, batch_index, batchsize)
            #sess.run(training_op, feed_dict={X:Xb, y:yb.reshape(-1,1)})
    best_theta=theta.eval(session=sess)
    #saver.save(sess, "./saved_models/final_minibatch.ckpt")

INFO:tensorflow:Restoring parameters from ./saved_models/final_minibatch.ckpt


KeyError: "The name 'Training_op:0' refers to a Tensor which does not exist. The operation, 'Training_op', exists but only has 0 outputs."

optimal theta for the scaled data:

In [33]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)